In [1]:
from qiskit import QuantumRegister, QuantumCircuit, ClassicalRegister

In [8]:
def initialize_program():
    qubit = QuantumRegister(2)
    A = qubit[0]
    B = qubit[1]
    print(A)
    print(B)
    print('\n')

    bit = ClassicalRegister(2)
    a = bit[0]
    b = bit[1]

    print('Classical')
    print(a)
    print(b)
    print('-'*24)

    qc = QuantumCircuit(qubit, bit)
    return A,B, a, b, qc 

print(initialize_program())


Qubit(QuantumRegister(2, 'q6'), 0)
Qubit(QuantumRegister(2, 'q6'), 1)


Classical
Clbit(ClassicalRegister(2, 'c1'), 0)
Clbit(ClassicalRegister(2, 'c1'), 1)
------------------------
(Qubit(QuantumRegister(2, 'q6'), 0), Qubit(QuantumRegister(2, 'q6'), 1), Clbit(ClassicalRegister(2, 'c1'), 0), Clbit(ClassicalRegister(2, 'c1'), 1), <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x12fcf6d60>)


In [11]:
def hash_bit(variable, hash_type, bit, qc):
    if hash_type == 'h':
        qc.x(variable)
    qc.measure(variable, bit)

In [10]:
from hello_qiskit import run_puzzle 
import numpy as np
puzzle = run_puzzle(12)

def setup_vars(A, B, qc):
    for line in puzzle.program:
        eval(line)


    

shots = 8192 

In [12]:
from qiskit import assemble, transpile
def calculate_P(backend):
    p = {}
    program = {}
    possibilities = ['vv', 'vh', 'hv', 'hh']
    for hashes in possibilities:
        A, B, a, b, program[hashes] = initialize_program()
        setup_vars(A, b, program[hashes])
        hash_bit(A, hashes[0], a, program[hashes])
        hash_bit(B, hashes[1], b, program[hashes])

        t_qes = transpile(list(program.values()),backend)
        qobj = assemble(t_qes, shots=shots)
        job = backend.run(qobj)
        
        for hashes in possibilities:
            stats = job.result().get_counts(program[hashes])
            p[hashes] = 0 
            for string in stats.keys():
                a = string[-1]
                b = string[-2]

                if a != b:
                    p[hashes] += stats[string]/shots 
    
    return p 


In [ ]:
puzzle.program 
from qiskit import Aer 
def bell_test(p):
    sum_p = sum(p.values())
    for hashes in p:
        bound = sum_p - p[hashes]
        print("Upper Bound of {} is {bound}".format(hashes, bound=bound))
        print("Value: {}".format(p[hashes]))
        if p[hashes] <= bound:
            print("The upper bound is obeyed")
        else:
            if p[hashes] - bound < 0.1:
                print("Over Upper Boundary by a little (rounding error)")
            else:
                print("Over Upper boundary by a lot")


backend = Aer.get_backend('aer_simulator')
p = calculate_P(backend)
print(p)
bell_test(p)
